In [ ]:
def runAnotherKeras(a, b,c,d):
    # mon cnn architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out  
    #epoque 100 normalizer
    batch_size = 128
    num_classes = 2
    epochs = 100
    seed=7
    # input image dimensions
    img_rows, img_cols = X_train.shape[1],X_train.shape[2]
    input_shape = (img_rows, img_cols, 3)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    cvscores = []
    for train, test in kfold.split(X, Y):
        model = Sequential()
        model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu', input_shape = input_shape))
        model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
        model.add(MaxPool2D(pool_size=(2,2)))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
        model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
        model.add(MaxPool2D(pool_size=(2,2)))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        model.add(Conv2D(filters = 86, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
        model.add(Conv2D(filters = 86, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
        model.add(MaxPool2D(pool_size=(2,2)))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1024, activation = "relu"))
        model.add(Dropout(0.5))
        model.add(Dense(512, activation = "relu"))
        model.add(Dropout(0.5))
        model.add(Dense(2, activation = "softmax"))
        optimizer = RMSprop(lr=0.001, decay=1e-6)
        model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
        model.fit(a,b,
                  batch_size=batch_size,
                  verbose=1,
                  epochs=epochs,
                  validation_data=(c,d),callbacks = [MetricsCheckpoint('logs')])
        score = model.evaluate(c,d, verbose=0)
        cvscores.append(score[1] * 100)
    print('\nKeras CNN #2 - accuracy:', score[1], '\n')
    y_pred = model.predict(c)
    map_characters = {0: 'IDC(-)', 1: 'IDC(+)'}
    print('\n', sklearn.metrics.classification_report(np.where(d > 0)[1], np.argmax(y_pred, axis=1), target_names=list(map_characters.values())), sep='') 
    Y_pred_classes = np.argmax(y_pred,axis = 1) 
    Y_true = np.argmax(Y_test,axis = 1) 
    confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
    plot_confusion_matrix(confusion_mtx, classes = list(map_characters.values())) 
    print("la moyenne est %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
runAnotherKeras(X_train, Y_train, X_test, Y_test)
plotKerasLearningCurve()